# Example on Calculation of Receiver Functions

This notebook demonstrates the process of calculating receiver functions using the deconvution methods in [rf](https://github.com/trichter/rf) package

The necessary functions are imported

In [1]:
# Receiver Function Calculation
import numpy as np
from rf import RFStream
from rf import IterMultipleComponents
from tqdm import tqdm
from rfsed.util import save_calculated_RF, read_raw_waveform_data


The path to save the output is defined

In [2]:
#------------------------------------------#
savedir=save_calculated_RF()
rffile = savedir + '00_rf_data.h5'

The waveform is pre-process including trimming the length of the data, filtering within defined frequency bands. 

The data is then rotated and deconvolved. In this example, the vertical component is deconvolved from the radial and transverse components. The receiver functions are saved to a file

In [3]:
eqdata = read_raw_waveform_data()
rfst = RFStream()
for stream3c in tqdm(IterMultipleComponents(eqdata, 'onset', 3)):
    if len(stream3c) != 3:
        continue    
    stream3c.trim2(-10, 40, 'onset')
    stream3c.detrend('linear')
    stream3c.detrend('demean')
    stream3c.taper(type = 'hann', max_percentage=0.07)
    stream3c.filter('bandpass', freqmin = 0.05, 
                  freqmax=1.25, corners=4, zerophase=True)
    stream3c.rf(deconvolve='iterative',  rotate='NE->RT', gauss=np.log(1.25))
    #stream3c.plot()
    rfst.extend(stream3c)
rfst.write(rffile, 'H5')

100%|██████████| 11/11 [00:01<00:00,  6.39it/s]
